In [25]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import concatenate
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
from helpers import TrainingMonitor
from helpers import Utils

In [27]:
output_path = "../output/"

In [28]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [29]:
db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [43]:
class INCEPTION:
    @staticmethod
    def conv_module(x, num_filter, size_filter, stride, reg, name=None):
        (convName, bnName, actName) = (None, None, None)
        if name is not None:
            convName = name + "_conv"
            bnName = name + "_bn"
            actName = name + "_act"
            
        x = Conv2D(num_filter, size_filter, strides=stride, padding="same", kernel_regularizer=l2(reg), name=convName)(x)
        x = Activation("relu", name=actName)(x)
        x = BatchNormalization(axis=-1, name=bnName)(x)
        return x

    @staticmethod
    def inception_module(x, num1x1, num3x3Reduce, num3x3, num5x5Reduce, num5x5, num1x1Proj, reg, stage):

        first = INCEPTION.conv_module(x, num1x1, 1, 1, reg, name=stage + "_1x1")

        second = INCEPTION.conv_module(x, num3x3Reduce, 1, 1, reg, name=stage + "_3x3_1")
        second = INCEPTION.conv_module(second, num3x3, 3, 1, reg, name=stage + "_3x3_2")

        third = INCEPTION.conv_module(x, num5x5Reduce, 1, 1, reg, name=stage + "_5x5_1")
        third = INCEPTION.conv_module(third, num5x5, 5, 1, reg, name=stage + "_5x5_2")

        fourth = MaxPooling2D(3, strides=1, padding="same", name=stage + "_pool_1")(x)
        fourth = INCEPTION.conv_module(fourth, num1x1Proj, 1, 1, reg, name=stage + "_pool_2")

        x = concatenate([first, second, third, fourth], axis=-1, name=stage + "_mixed")
        return x
    
    @staticmethod
    def build(include_top=True, reg=0.0005):
        inputs = Input(shape=x_train_rgbmean.shape[1:])
        
        x = INCEPTION.inception_module(inputs, 32, 32, 32, 16, 16, 32, reg, "block1")
        x = INCEPTION.inception_module(x, 64, 64, 64, 32, 32, 64, reg, "block2")
        x = INCEPTION.inception_module(x, 64, 64, 64, 32, 32, 64, reg, "block3")
        
        if include_top:
            x = AveragePooling2D((8, 8))(x)
            x = Dropout(0.5, name="do")(x)
            x = Flatten(name="flatten")(x)
            x = Dense(10, kernel_regularizer=l2(reg), name="labels")(x)
            x = Activation("softmax", name="softmax")(x)
            
        model = Model(inputs, x)
        return model

In [44]:
model = INCEPTION.build(include_top=True, reg=0.0005)

In [1]:
model.summary()

NameError: name 'model' is not defined

In [46]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-inception-v1.png', show_shapes=True, show_layer_names=True)

In [47]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
#filepath=output_path + "progress/inception-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
print(os.getpid())
filepath=output_path + "progress/inception-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=5)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

8916


In [50]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=5,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

Train on 33500 samples, validate on 16500 samples
Epoch 1/5
33500/33500 [==============================] - 95s 3ms/step - loss: 0.1813 - acc: 0.9473 - val_loss: 0.1882 - val_acc: 0.9443
Epoch 2/5
33500/33500 [==============================] - 95s 3ms/step - loss: 0.1766 - acc: 0.9490 - val_loss: 0.1799 - val_acc: 0.9472
Epoch 3/5
33500/33500 [==============================] - 95s 3ms/step - loss: 0.1743 - acc: 0.9501 - val_loss: 0.1861 - val_acc: 0.9464
Epoch 4/5
33500/33500 [==============================] - 95s 3ms/step - loss: 0.1714 - acc: 0.9516 - val_loss: 0.1843 - val_acc: 0.9467
Epoch 5/5
33500/33500 [==============================] - 95s 3ms/step - loss: 0.1695 - acc: 0.9522 - val_loss: 0.1747 - val_acc: 0.9503


In [51]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 95.22%; Val: 95.03%; Test: 95.02%


In [52]:
model_json = model.to_json()
with open(output_path + "cifar_model_inception_v1_95.02.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(output_path + "cifar_weight_inception_v1_95.02.hdf5")

In [53]:
json_file = open(output_path + 'cifar_model_inception_v1_95.02.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(output_path + "cifar_weight_inception_v1_95.02.hdf5")